## Partie 2 : Implémenter pas à pas le pipeline

Écrire un script Python qui :

- charge les données d’entrée dans des DataFrames pandas (CSV/JSON/…),
- applique les transformations définies,
- sauvegarde le résultat dans un fichier CSV et dans une base de données conformément au cahier - des charges.

In [49]:
import os
import sqlite3
import pandas as pd
import yaml
from datetime import datetime
from pathlib import Path
from IPython.display import display, Markdown

### Configuration (settings.yaml)

In [50]:
def load_settings(path="../../settings.yaml"):
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

In [51]:
cfg = load_settings()
in_dir = cfg.get("input_dir", "../../data/march-input")
out_dir = cfg.get("output_dir", "../../data/out")
db_path = cfg.get("db_path", "../../data/sales_db.db")
sep = cfg.get("csv_sep", ";")
enc = cfg.get("csv_encoding", "utf-8")
ffmt = cfg.get("csv_float_format", "%.2f")
Path(out_dir).mkdir(parents=True, exist_ok=True)
display(
    Markdown(
        f"Input dir: `{in_dir}`\n"
        f"Output dir: `{out_dir}`\n"
        f"DB (SQLite) : `{db_path}`"
    )
)

Input dir: `../../data/march-input`
Output dir: `../../data/out`
DB (SQLite) : `../../data/out/sales.db`

### 1- Charger les données d’entrée dans des DataFrames pandas (CSV/JSON/…)

Charger les informations sur les clients `customers.csv`

Syntaxe commande : `out_df = pd.read_csv("input path")`

In [52]:
customers_path = os.path.join(in_dir, "customers.csv")
print(customers_path)
# customers_path = ("/chemin vers /customers.csv")
if not os.path.exists(customers_path):
    display(Markdown(f" Fichier manquant : `{customers_path}`."))
else:
    customers = pd.read_csv(customers_path)
    display(customers.head(30))
    display(Markdown(f"Taille: {customers.shape}"))

../../data/march-input/customers.csv


,customer_id,first_name,last_name,email,city,is_active
0,C0001,User1,Test1,user1@example.com,Nantes,yes
1,C0002,User2,Test2,user2@example.com,Toulouse,yes
2,C0003,User3,Test3,user3@example.com,Bordeaux,y
3,C0004,User4,Test4,user4@example.com,Bordeaux,true
4,C0005,User5,Test5,user5@example.com,Lyon,true
5,C0006,User6,Test6,user6@example.com,Marseille,false
6,C0007,User7,Test7,user7@example.com,Toulouse,true
7,C0008,User8,Test8,user8@example.com,Marseille,false
8,C0009,User9,Test9,user9@example.com,Toulouse,false
9,C0010,User10,Test10,user10@example.com,Bordeaux,true


Taille: (800, 6)

Charger les informations sur les clients `refunds.csv`

Syntaxe commande : `out_df = pd.read_csv("input path")`

In [53]:
refunds_path = os.path.join(in_dir, "refunds.csv")
# refunds_path = ("/chemin vers /refunds.csv")
if not os.path.exists(refunds_path):
    display(Markdown(f" Fichier manquant : `{refunds_path}`."))
else:
    refunds = pd.read_csv(refunds_path)
    display(refunds.head())
    display(Markdown(f"Taille: {refunds.shape}"))

,refund_id,order_id,amount,reason,created_at
0,R000001,O202503010089,error,delay,2025-03-01 14:03:41
1,R000002,O202503010038,-8.89,gesture,2025-03-01 22:16:56
2,R000003,O202503010008,again,item_issue,2025-03-01 20:06:25
3,R000004,O202503010073,-2.47,coupon,2025-03-01 20:02:46
4,R000005,O202503010005,-3.83,gesture,2025-03-01 09:58:15


Taille: (1122, 5)

Charger les informations sur les commandes `orders_*.json`

Syntaxe commande : `df = pd.read_json("data.json")`

In [54]:
order_path = os.path.join(in_dir, "orders_2025-03-01.json")
# order_path = ("/Users/estherchabi/Documents/fr/Caplogy/2025-2026/Simplon/Inter UI ARA HDF /Esther_Phase0/Brief1111/projet_python/data/march-input/orders_2025-03-01.json")

if not os.path.exists(order_path):
    display(Markdown(f" Fichier manquant : `{order_path}`."))
else:
    order = pd.read_json(order_path)
    display(order.head())
    display(Markdown(f"Taille: {order.shape}"))

,order_id,customer_id,channel,created_at,payment_status,items
0,O202503010001,C0793,app,2025-03-01 20:36:44,pending,"[{'sku': 'SKU001', 'qty': 4, 'unit_price': 24.9}]"
1,O202503010001,C0676,web,2025-03-01 11:30:49,paid,"[{'sku': 'SKU042', 'qty': 4, 'unit_price': -7...."
2,O202503010003,C0642,web,2025-03-01 07:27:00,paid,"[{'sku': 'SKU014', 'qty': 1, 'unit_price': 5.0}]"
3,O202503010004,C0283,web,2025-03-01 14:28:46,pending,"[{'sku': 'SKU024', 'qty': 2, 'unit_price': 4.0}]"
4,O202503010005,C0571,web,2025-03-01 22:29:42,paid,"[{'sku': 'SKU001', 'qty': 1, 'unit_price': 2.5}]"


Taille: (103, 6)

In [55]:
# day=1
# order_path = os.path.join(in_dir, f"orders_2025-03-{day:02d}.json")
# display(Markdown(f"Order dir: `{order_path}`\n"))

In [56]:
liste = []

for day in range(1, 32):
    order_path = os.path.join(in_dir, f"orders_2025-03-{day:02d}.json")

    if not os.path.exists(order_path):
        display(Markdown(f" Fichier manquant : `{order_path}`."))
        continue
    else:
        order = pd.read_json(order_path)
        # display(Markdown(f"Taille: {order.shape}")) # Taille: (103, 6)

    liste.append(order)
    # display(Markdown(f"Taille : {liste}"))

orders = pd.concat(liste)

display(orders.head())
display(Markdown(f"Taille: {orders.shape}"))  # Taille: (103x31, 6) = (3193, 6)

,order_id,customer_id,channel,created_at,payment_status,items
0,O202503010001,C0793,app,2025-03-01 20:36:44,pending,"[{'sku': 'SKU001', 'qty': 4, 'unit_price': 24.9}]"
1,O202503010001,C0676,web,2025-03-01 11:30:49,paid,"[{'sku': 'SKU042', 'qty': 4, 'unit_price': -7...."
2,O202503010003,C0642,web,2025-03-01 07:27:00,paid,"[{'sku': 'SKU014', 'qty': 1, 'unit_price': 5.0}]"
3,O202503010004,C0283,web,2025-03-01 14:28:46,pending,"[{'sku': 'SKU024', 'qty': 2, 'unit_price': 4.0}]"
4,O202503010005,C0571,web,2025-03-01 22:29:42,paid,"[{'sku': 'SKU001', 'qty': 1, 'unit_price': 2.5}]"


Taille: (3193, 6)

### Appliquer les transformations définies dans le cahier de charge

#### Transformation client:
- Exclure les clients inactifs (is_active = false)

La colonne is_active doit être booléenne (j'ai écris une petite fonction qui vérifie que la colonne est booleenne)



In [57]:
def controle_bool(v):
    if isinstance(v, bool):
        return v
    if isinstance(v, (int, float)):
        return bool(v)
    if v is None:
        return False
    s = str(v).strip().lower()
    return s in ("1", "true", "yes", "y", "t")

In [58]:
customers["is_active"] = customers["is_active"].apply(controle_bool)
customers = customers.astype({"customer_id": "string", "city": "string"})
display(Markdown("Affichage clients (après nettoyage)"))
display(customers.head(30))
display(Markdown(f"Taille: {customers.shape}"))

Affichage clients (après nettoyage)

,customer_id,first_name,last_name,email,city,is_active
0,C0001,User1,Test1,user1@example.com,Nantes,True
1,C0002,User2,Test2,user2@example.com,Toulouse,True
2,C0003,User3,Test3,user3@example.com,Bordeaux,True
3,C0004,User4,Test4,user4@example.com,Bordeaux,True
4,C0005,User5,Test5,user5@example.com,Lyon,True
5,C0006,User6,Test6,user6@example.com,Marseille,False
6,C0007,User7,Test7,user7@example.com,Toulouse,True
7,C0008,User8,Test8,user8@example.com,Marseille,False
8,C0009,User9,Test9,user9@example.com,Toulouse,False
9,C0010,User10,Test10,user10@example.com,Bordeaux,True


Taille: (800, 6)

#### Transformation remboursements (refunds)

Pas de transformations précisées explicitement dans le cachier de charge, par contre on me demande de : agréger les remboursements par commande. Il faudra donc que je m'assure que la colonne `amount` contient des montants numériques décimaux propres, afin de permettre une agrégation fiable

In [59]:
# refunds = pd.read_csv(refunds_path)
refunds["amount"] = pd.to_numeric(refunds["amount"], errors="coerce").fillna(0.0)
refunds["created_at"] = refunds["created_at"].astype("string")
display(Markdown("Aperçu remboursements (après coercition numérique)"))
display(refunds.head())
display(Markdown(f"Taille: {refunds.shape}"))

Aperçu remboursements (après coercition numérique)

,refund_id,order_id,amount,reason,created_at
0,R000001,O202503010089,0.00,delay,2025-03-01 14:03:41
1,R000002,O202503010038,-8.89,gesture,2025-03-01 22:16:56
2,R000003,O202503010008,0.00,item_issue,2025-03-01 20:06:25
3,R000004,O202503010073,-2.47,coupon,2025-03-01 20:02:46
4,R000005,O202503010005,-3.83,gesture,2025-03-01 09:58:15


Taille: (1122, 5)

### Transformation commandes : orders


- Conserver uniquement les commandes payées (payment_status = 'paid')
- Écarter toute ligne d’article avec prix unitaire négatif (et consigner
ces rejets)
- Dédupliquer sur order_id (garder la première occurrence).

In [60]:
# Filtrer les commandes payées (payment_status == 'paid')
# On peut aussi ecrire une fonction qui verifie que le status est bien 'paid'
# ex: p,ok,yes => paid
# def controle_paid(v):
#     v = v.lower()
#     if v in ['p', 'ok', 'yes']:
#         return 'paid'
#     return 'other'
# orders["payment_status"] = orders["payment_status"].apply(controle_paid)
ln_initial = len(orders)
orders = orders[orders["payment_status"] == "paid"].copy()
ln_final = len(orders)
display(Markdown(f"Filtrage payées : {ln_initial} → {ln_final}"))
display(orders.head())

Filtrage payées : 3193 → 2900

,order_id,customer_id,channel,created_at,payment_status,items
1,O202503010001,C0676,web,2025-03-01 11:30:49,paid,"[{'sku': 'SKU042', 'qty': 4, 'unit_price': -7...."
2,O202503010003,C0642,web,2025-03-01 07:27:00,paid,"[{'sku': 'SKU014', 'qty': 1, 'unit_price': 5.0}]"
4,O202503010005,C0571,web,2025-03-01 22:29:42,paid,"[{'sku': 'SKU001', 'qty': 1, 'unit_price': 2.5}]"
5,O202503010006,C0704,web,2025-03-01 09:24:19,paid,"[{'sku': 'SKU039', 'qty': 1, 'unit_price': 9.9..."
6,O202503010007,C0464,app,2025-03-01 15:50:48,paid,"[{'sku': 'SKU018', 'qty': 1, 'unit_price': 24...."


Ecarter toute ligne d’article dont le prix unitaire est négatif et journaliser ces rejets.

Le prix unitaire est porté par la structure items : il faut d’abord aplatir/“exploser” cette colonne pour obtenir une ligne par article, puis appliquer le filtre.
Aplatissement possible via `pandas.json_normalize` :
items = pd.json_normalize(orders["items"])

In [61]:
orders2 = orders
display(Markdown("Avant explosion des items"))
display(orders2.head())
# display(Markdown(f"Colonnes: {list(orders2.columns)[:12]}

Avant explosion des items

,order_id,customer_id,channel,created_at,payment_status,items
1,O202503010001,C0676,web,2025-03-01 11:30:49,paid,"[{'sku': 'SKU042', 'qty': 4, 'unit_price': -7...."
2,O202503010003,C0642,web,2025-03-01 07:27:00,paid,"[{'sku': 'SKU014', 'qty': 1, 'unit_price': 5.0}]"
4,O202503010005,C0571,web,2025-03-01 22:29:42,paid,"[{'sku': 'SKU001', 'qty': 1, 'unit_price': 2.5}]"
5,O202503010006,C0704,web,2025-03-01 09:24:19,paid,"[{'sku': 'SKU039', 'qty': 1, 'unit_price': 9.9..."
6,O202503010007,C0464,app,2025-03-01 15:50:48,paid,"[{'sku': 'SKU018', 'qty': 1, 'unit_price': 24...."


In [62]:
orders2 = orders2.explode("items", ignore_index=True)
display(orders.head())
items = pd.json_normalize(orders2["items"]).add_prefix("item_")
display(items)
orders2 = pd.concat([orders2.drop(columns=["items"]), items], axis=1)
display(Markdown("Après explosion des items"))
display(orders2.head())
display(Markdown(f"Colonnes: {list(orders2.columns)[:12]} ..."))

,order_id,customer_id,channel,created_at,payment_status,items
1,O202503010001,C0676,web,2025-03-01 11:30:49,paid,"[{'sku': 'SKU042', 'qty': 4, 'unit_price': -7...."
2,O202503010003,C0642,web,2025-03-01 07:27:00,paid,"[{'sku': 'SKU014', 'qty': 1, 'unit_price': 5.0}]"
4,O202503010005,C0571,web,2025-03-01 22:29:42,paid,"[{'sku': 'SKU001', 'qty': 1, 'unit_price': 2.5}]"
5,O202503010006,C0704,web,2025-03-01 09:24:19,paid,"[{'sku': 'SKU039', 'qty': 1, 'unit_price': 9.9..."
6,O202503010007,C0464,app,2025-03-01 15:50:48,paid,"[{'sku': 'SKU018', 'qty': 1, 'unit_price': 24...."


,item_sku,item_qty,item_unit_price
0,SKU042,4,-7.5
1,SKU042,4,-7.5
2,SKU005,5,12.5
3,SKU014,1,5.0
4,SKU001,1,2.5
...,...,...,...
7260,SKU002,1,2.5
7261,SKU031,2,4.0
7262,SKU039,4,8.0
7263,SKU012,2,15.0


Après explosion des items

,order_id,customer_id,channel,created_at,payment_status,item_sku,item_qty,item_unit_price
0,O202503010001,C0676,web,2025-03-01 11:30:49,paid,SKU042,4,-7.5
1,O202503010001,C0676,web,2025-03-01 11:30:49,paid,SKU042,4,-7.5
2,O202503010001,C0676,web,2025-03-01 11:30:49,paid,SKU005,5,12.5
3,O202503010003,C0642,web,2025-03-01 07:27:00,paid,SKU014,1,5.0
4,O202503010005,C0571,web,2025-03-01 22:29:42,paid,SKU001,1,2.5


Colonnes: ['order_id', 'customer_id', 'channel', 'created_at', 'payment_status', 'item_sku', 'item_qty', 'item_unit_price'] ...

In [63]:
neg_mask = orders2["item_unit_price"] < 0
n_neg = int(neg_mask.sum())
display(Markdown(f"Lignes prix négatifs : {n_neg}"))
if n_neg > 0:
    rejects_items = orders2.loc[neg_mask].copy()
    rejects_path = os.path.join(out_dir, "rejects_items.csv")
    rejects_items.to_csv(rejects_path, index=False, encoding=enc)
    display(Markdown(f" Rejets sauvegardés : `{rejects_path}`"))
    orders2 = orders2.loc[~neg_mask].copy()
display(orders2.head())

Lignes prix négatifs : 69

 Rejets sauvegardés : `../../data/out/rejects_items.csv`

,order_id,customer_id,channel,created_at,payment_status,item_sku,item_qty,item_unit_price
2,O202503010001,C0676,web,2025-03-01 11:30:49,paid,SKU005,5,12.5
3,O202503010003,C0642,web,2025-03-01 07:27:00,paid,SKU014,1,5.0
4,O202503010005,C0571,web,2025-03-01 22:29:42,paid,SKU001,1,2.5
5,O202503010006,C0704,web,2025-03-01 09:24:19,paid,SKU039,1,9.9
6,O202503010006,C0704,web,2025-03-01 09:24:19,paid,SKU037,4,15.0


Dédupliquer sur order_id (garder la première occurrence).


In [64]:
before = len(orders2)
orders3 = orders2.sort_values(["order_id", "created_at"]).drop_duplicates(
    subset=["order_id"], keep="first"
)
after = len(orders3)
display(Markdown(f"Déduplication : **{before} → {after}**"))
display(orders3.head())

Déduplication : **7196 → 2811**

,order_id,customer_id,channel,created_at,payment_status,item_sku,item_qty,item_unit_price
2,O202503010001,C0676,web,2025-03-01 11:30:49,paid,SKU005,5,12.5
3,O202503010003,C0642,web,2025-03-01 07:27:00,paid,SKU014,1,5.0
4,O202503010005,C0571,web,2025-03-01 22:29:42,paid,SKU001,1,2.5
5,O202503010006,C0704,web,2025-03-01 09:24:19,paid,SKU039,1,9.9
9,O202503010007,C0464,app,2025-03-01 15:50:48,paid,SKU018,1,24.9


### Aggregations et jointures

Sortie attendue : 
Colonnes: date;city;`channel`;orders_count;unique_customers;`items_sold`;`gross_revenue_eur`;refunds_eur;net_revenue_eur

On peut commencer avec la table oders
- Calculer une colonne line_gross = item_qty x item_unit_price
- Agreger les données selon (order_id, customer_id, channel, created_at) en totalisant items_sold et gross_revenue_eur

In [65]:
orders3["line_gross"] = orders3["item_qty"] * orders3["item_unit_price"]
per_order = orders3.groupby(
    ["order_id", "customer_id", "channel", "created_at"], as_index=False
).agg(items_sold=("item_qty", "sum"), gross_revenue_eur=("line_gross", "sum"))
display(Markdown("Aperçu `per_order`"))
display(per_order.head())
display(Markdown(f"Taille: {per_order.shape}"))

Aperçu `per_order`

,order_id,customer_id,channel,created_at,items_sold,gross_revenue_eur
0,O202503010001,C0676,web,2025-03-01 11:30:49,5,62.5
1,O202503010003,C0642,web,2025-03-01 07:27:00,1,5.0
2,O202503010005,C0571,web,2025-03-01 22:29:42,1,2.5
3,O202503010006,C0704,web,2025-03-01 09:24:19,1,9.9
4,O202503010007,C0464,app,2025-03-01 15:50:48,1,24.9


Taille: (2811, 6)

 Jointure avec la table clients
 
 Colonnes: date;`city`;`channel`;orders_count;unique_customers;`items_sold`;`gross_revenue_eur`;refunds_eur;net_revenue_eur


Ici on réalise une jointure entre la table des commandes per_order et la table customers afin d’enrichir chaque commande avec les informations client (city, is_active) à partir de la clé commune customer_id. 

Le paramètre how="left" indique qu’il s’agit d’une jointure gauche : on garde toutes les lignes de per_order et on ajoute les données de customers quand un customer_id correspondant existe (sinon on obtient des valeurs nulles). 

En pandas (comme en SQL), on dispose aussi d’autres types de jointures :
- inner : ne conserve que les lignes présentes dans les deux tables (intersection),
- right : symétrique de left, on garde toutes les lignes de la table de droite,
- outer : conserve toutes les lignes des deux tables, en complétant avec des valeurs nulles lorsqu’il manque une correspondance.

In [66]:
len_init = len(per_order)
per_order = per_order.merge(
    customers[["customer_id", "city", "is_active"]], on="customer_id", how="left"
)
per_order = per_order[
    per_order["is_active"] == True
].copy()  # Important pour respecter le cahier de charge
ln_aft = len(per_order)
display(Markdown(f"Après jointure+filtre actifs : **{len_init} → {ln_aft}**"))
display(per_order.head())

Après jointure+filtre actifs : **2811 → 2471**

,order_id,customer_id,channel,created_at,items_sold,gross_revenue_eur,city,is_active
0,O202503010001,C0676,web,2025-03-01 11:30:49,5,62.5,Marseille,True
1,O202503010003,C0642,web,2025-03-01 07:27:00,1,5.0,Toulouse,True
2,O202503010005,C0571,web,2025-03-01 22:29:42,1,2.5,Toulouse,True
4,O202503010007,C0464,app,2025-03-01 15:50:48,1,24.9,Nantes,True
5,O202503010008,C0317,app,2025-03-01 20:56:15,2,30.0,Marseille,True


Extraire la **date** depuis `created_at`

La fonction `to_date()` renvoie la date au format texte "YYYY-MM-DD"

In [67]:
def to_date(s):
    s = str(s)
    for fmt in ("%Y-%m-%d %H:%M:%S", "%Y-%m-%d"):
        try:
            return datetime.strptime(s, fmt).date().isoformat()
        except ValueError:
            continue
    raise ValueError(f"Format de date non reconnu: {s}")

In [68]:
per_order["order_date"] = per_order["created_at"].apply(to_date)
display(per_order[["order_id", "created_at", "order_date"]].head())

,order_id,created_at,order_date
0,O202503010001,2025-03-01 11:30:49,2025-03-01
1,O202503010003,2025-03-01 07:27:00,2025-03-01
2,O202503010005,2025-03-01 22:29:42,2025-03-01
4,O202503010007,2025-03-01 15:50:48,2025-03-01
5,O202503010008,2025-03-01 20:56:15,2025-03-01


 Jointure avec la table refunds
 
 Colonnes: date;`city`;`channel`;orders_count;unique_customers;`items_sold`;`gross_revenue_eur`;`refunds_eur`;net_revenue_eur


In [69]:
refunds_sum = (
    refunds.groupby("order_id", as_index=False)["amount"]
    .sum()
    .rename(columns={"amount": "refunds_eur"})
)  # Somme des remboursements par order_id (commande)
per_order = per_order.merge(refunds_sum, on="order_id", how="left").fillna(
    {"refunds_eur": 0.0}
)
display(per_order.head())

,order_id,customer_id,channel,created_at,items_sold,gross_revenue_eur,city,is_active,order_date,refunds_eur
0,O202503010001,C0676,web,2025-03-01 11:30:49,5,62.5,Marseille,True,2025-03-01,0.00
1,O202503010003,C0642,web,2025-03-01 07:27:00,1,5.0,Toulouse,True,2025-03-01,-3.13
2,O202503010005,C0571,web,2025-03-01 22:29:42,1,2.5,Toulouse,True,2025-03-01,-35.42
3,O202503010007,C0464,app,2025-03-01 15:50:48,1,24.9,Nantes,True,2025-03-01,0.00
4,O202503010008,C0317,app,2025-03-01 20:56:15,2,30.0,Marseille,True,2025-03-01,-3.69


### Sauvegarde intermédiaire dans SQLite (`orders_clean`)

In [70]:
conn = sqlite3.connect(db_path)
per_order_save = per_order[
    [
        "order_id",
        "customer_id",
        "city",
        "channel",
        "order_date",
        "items_sold",
        "gross_revenue_eur",
    ]
].copy()
per_order_save.to_sql("orders_clean", conn, if_exists="replace", index=False)
display(Markdown("Table `orders_clean` sauvegardée dans SQLite"))
conn.close()

Table `orders_clean` sauvegardée dans SQLite

### Agrégat final (date, ville, canal) + **net**

In [71]:
agg = per_order.groupby(["order_date", "city", "channel"], as_index=False).agg(
    orders_count=("order_id", "nunique"),
    unique_customers=("customer_id", "nunique"),
    items_sold=("items_sold", "sum"),
    gross_revenue_eur=("gross_revenue_eur", "sum"),
    refunds_eur=("refunds_eur", "sum"),
)
agg["net_revenue_eur"] = agg["gross_revenue_eur"] + agg["refunds_eur"]
agg = (
    agg.rename(columns={"order_date": "date"})
    .sort_values(["date", "city", "channel"])
    .reset_index(drop=True)
)
display(agg.head())
display(Markdown(f"Taille: {agg.shape}"))

,date,city,channel,orders_count,unique_customers,items_sold,gross_revenue_eur,refunds_eur,net_revenue_eur
0,2025-03-01,Bordeaux,app,4,4,8,136.8,-25.55,111.25
1,2025-03-01,Bordeaux,web,6,6,13,195.2,-39.73,155.47
2,2025-03-01,Lille,app,4,4,13,224.2,-24.47,199.73
3,2025-03-01,Lille,web,3,3,12,117.5,-14.23,103.27
4,2025-03-01,Lyon,app,7,7,16,159.5,-10.86,148.64


Taille: (491, 9)

### Écriture finale en base (`daily_city_sales`) + exports CSV

In [72]:
conn = sqlite3.connect(db_path)
agg.to_sql("daily_city_sales", conn, if_exists="replace", index=False)
conn.close()
display(Markdown("Table `daily_city_sales` écrite dans SQLite"))
# Exports CSV
for d, sub in agg.groupby("date"):
    out_path = os.path.join(out_dir, f"daily_summary_{d.replace('-','')}.csv")
    sub[
        [
            "date",
            "city",
            "channel",
            "orders_count",
            "unique_customers",
            "items_sold",
            "gross_revenue_eur",
            "refunds_eur",
            "net_revenue_eur",
        ]
    ].to_csv(out_path, index=False, sep=sep, encoding=enc, float_format=ffmt)
all_path = os.path.join(out_dir, "daily_summary_all.csv")
agg.to_csv(all_path, index=False, sep=sep, encoding=enc, float_format=ffmt)
display(Markdown(f"Exports CSV dans `{out_dir}`"))

Table `daily_city_sales` écrite dans SQLite

Exports CSV dans `../../data/out`